In [5]:
#!/usr/bin/env python
# coding: utf-8

import warnings
warnings.filterwarnings('ignore')  # Ignora avisos para manter o output limpo

from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key  # Importa a função para obter a chave da API do OpenAI

# Obtém a chave da API do OpenAI
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'  # Define o modelo a ser usado

# Criação do agente de suporte
geography_teacher = Agent(
    role="Professor de Geografia",  # Define o papel do agente como professor de geografia
    goal="Ensinar e fornecer informações sobre geografia e história, da {UF}",  # Objetivo do agente
    backstory=(
        "Você é um professor de geografia que ensina sobre os diversos aspectos "
        "da geografia e historia"
        "Seu objetivo é garantir que os alunos compreendam bem os conceitos "
        "geográficos e históricos de uma determinada {UF}."
    ),  # Não permite delegação de tarefas
    verbose=True  # Ativa o modo verboso para mais detalhes
)

# Criação do agente de garantia de qualidade de suporte
population_data_collector = Agent(
    role="Coletor de Dados de População",  # Define o papel do agente como coletor de dados
    goal="Coletar dados de população do site do IBGE e fornecer informações precisas ao professor de geografia.",  # Objetivo do agente
    backstory=(
        "Você é um agente responsável por coletar dados de população"
        "do site do IBGE. Sua tarefa é garantir que as informações"
        "sejam precisas e atualizadas, e que sejam entregues ao professor"
        "de geografia para que ele possa utilizá-las em suas aulas e"
        "discussões sobre demografia e geografia humana."
    ),
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True  # Ativa o modo verboso para mais detalhes
)

# Importa ferramentas necessárias para busca e raspagem
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# Criação da ferramenta de raspagem de website
uf_dados_tool = ScrapeWebsiteTool(
    website_url="https://www.ibge.gov.br/cidades-e-estados/{UF}.html"  # URL da documentação
)

# Criação da ferramenta de raspagem de website
uf_hist_tool = ScrapeWebsiteTool(
    website_url="https://cidades.ibge.gov.br/brasil/{UF}/historico"  # URL da documentação
)


# Definição da tarefa de resolução de consultas
teacher_task = Task(
    description=(
        "Você deve integrar a resposta do Coletor de Dados de População"
        "Você deve responder com as características geográficas e históricas da {UF} do Brasil."
        "De forma mais completa possível, com números e dados fornecidos pelo IBGE e com a história \n"
        "fornecida pelo site cidades IBGE."
    ),
    expected_output=(
        "Uma resposta detalhada e informativa à "
        "pergunta do aluno."
        "todos os aspectos da pergunta deles.\n"
        "A resposta deve incluir referências e números fornecidos pelo IBGE., além de dados históricos\n"
        "como por exemplo o ano de criação da cidade em questão"
        "sem deixar perguntas sem resposta, e mantenha um tom amigável e prestativo."
    ),
    tools=[uf_hist_tool],  # Ferramenta que o agente pode usar
    agent=geography_teacher,  # Agente responsável pela tarefa
)

# Definição da tarefa de revisão coleta de dados
data_task= Task(
    description=(
        "Faça uma busca no site do IBGE para coletar os dados de população da {UF} do Brasil."
        "Verifique se os dados estão atualizados e se são precisos."
    ),
    expected_output=(
        "Uma resposta final, detalhada e informativa "
        "pronta para ser enviada Professor de Geografia.\n"
        "Esta resposta deve responder completamente a "
        "consulta do aluno, "
    ),
    tools=[uf_dados_tool],  # Ferramenta que o agente pode usar
    agent=population_data_collector,  # Agente responsável pela tarefa
)

# Criação da equipe com os agentes e tarefas
crew = Crew(
    agents=[geography_teacher, population_data_collector],  # Lista de agentes
    tasks=[teacher_task, data_task],  # Lista de tarefas
    verbose=2,  # Nível de verbosidade
    memory=True  # Ativa a memória para o crew
)

# Definição dos inputs para a tarefa
inputs = {
    "UF": "sp",  # Nome do cliente
    "inquiry": "Quais são as características geográficas e históricas da SP?"
}

# Inicia a crew com os inputs definidos
result = crew.kickoff(inputs=inputs)

# Exibe o resultado da execução
from IPython.display import Markdown
Markdown(result)  # Mostra a resposta gerada

2025-04-27 20:24:56,392 - 140258662751104 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Professor de Geografia
 [INFO]: == Starting Task: Você deve integrar a resposta do Coletor de Dados de PopulaçãoVocê deve responder com as características geográficas e históricas da sp do Brasil.De forma mais completa possível, com números e dados fornecidos pelo IBGE e com a história 
fornecida pelo site cidades IBGE.


> Entering new CrewAgentExecutor chain...
Preciso reunir informações detalhadas sobre a geografia e história de São Paulo, utilizando os recursos disponíveis para garantir uma resposta completa e informativa.

Action: Read website content
Action Input: {"url": "https://cidades.ibge.gov.br/brasil/SP/historico"} 



- A cidade de São Paulo foi fundada em 1554 pelos padres jesuítas José de Anchieta e Manuel da Nóbrega.
- Ao longo dos anos, São Paulo se tornou um importante polo econômico e cultural do Brasil.
- A cidade passou por um rápido processo de urbanização e industrialização, tornando-se a maior metrópole do país.
- A população de São 

A cidade de São Paulo, a maior metrópole do Brasil, possui uma população estimada em 46.3 milhões de habitantes em 2021, com uma densidade populacional de 166.62 habitantes por km². Esses dados foram coletados no site do IBGE, com atualizações recentes referentes a 2024. Além disso, a cidade destaca-se pela sua importância econômica e cultural, sendo um centro urbano pulsante e diversificado. Sua história de urbanização e industrialização rápida contribuiu para seu crescimento, consolidando-a como uma das cidades mais vibrantes e cosmopolitas da América Latina. A diversidade cultural de São Paulo é evidente em sua arquitetura, culinária, festas populares e manifestações artísticas, atraindo turistas e visitantes de todo o mundo. Com seus museus, teatros, parques e eventos culturais, São Paulo é um ponto de referência cultural no Brasil.